In [11]:
import yfinance as yf
import pandas as pd
import numpy as np
from pymongo import MongoClient
import datetime
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from talib import abstract
import math
## This is to Convert a string representation of list into list 
import ast
from functools import partial


In [19]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)
import backtrader as bt

In [5]:
stock='TCS'
startdate='2022-01-01'
enddate='2022-01-01'
exchange=['.NS','.BO']
dfo = yf.download(stock+exchange[0],start=startdate,interval='1h')

[*********************100%***********************]  1 of 1 completed


In [25]:
df=dfo.copy()

In [32]:
# Create a Stratey
class TestStrategy(bt.Strategy):

    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close
        self.dataopen = self.datas[0].open
        self.datahigh = self.datas[0].high
        self.datalow = self.datas[0].low

        # To keep track of pending orders
        self.order = None

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log('BUY EXECUTED, %.2f' % order.executed.price)
            elif order.issell():
                self.log('SELL EXECUTED, %.2f' % order.executed.price)

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None

    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        # Check if we are in the market
        if not self.position:

            # Not yet ... we MIGHT BUY if ...
            if self.dataclose[0] < self.dataclose[-1]:
                    # current close less than previous close

                    if self.dataclose[-1] < self.dataclose[-2]:
                        # previous close less than the previous close

                        # BUY, BUY, BUY!!! (with default parameters)
                        self.log('BUY CREATE, %.2f' % self.dataclose[0])

                        # Keep track of the created order to avoid a 2nd order
                        self.order = self.buy()

        else:

            # Already in the market ... we might sell
            if len(self) >= (self.bar_executed + 5):
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('SELL CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()

In [33]:
if __name__ == '__main__':
    cerebro = bt.Cerebro()
    # Add a strategy
    cerebro.addstrategy(TestStrategy)
    
    # Create a Data Feed
    data = bt.feeds.PandasData(dataname=df)
    
    # Add the Data Feed to Cerebro
    cerebro.adddata(data)

    # Set our desired cash start
    cerebro.broker.setcash(100000.0)
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    cerebro.run()

    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 100000.00
2022-01-03, Close, 3800.15
2022-01-03, Close, 3818.95
2022-01-03, Close, 3803.05
2022-01-03, Close, 3819.50
2022-01-03, Close, 3824.90
2022-01-03, Close, 3818.00
2022-01-03, Close, 3817.75
2022-01-03, BUY CREATE, 3817.75
2022-01-04, BUY EXECUTED, 3831.95
2022-01-04, Close, 3848.00
2022-01-04, Close, 3844.05
2022-01-04, Close, 3845.00
2022-01-04, Close, 3871.75
2022-01-04, Close, 3880.60
2022-01-04, Close, 3884.45
2022-01-04, SELL CREATE, 3884.45
2022-01-04, SELL EXECUTED, 3884.10
2022-01-04, Close, 3878.70
2022-01-05, Close, 3847.85
2022-01-05, BUY CREATE, 3847.85
2022-01-05, BUY EXECUTED, 3847.55
2022-01-05, Close, 3817.90
2022-01-05, Close, 3839.00
2022-01-05, Close, 3849.60
2022-01-05, Close, 3858.00
2022-01-05, Close, 3862.70
2022-01-05, Close, 3856.00
2022-01-05, SELL CREATE, 3856.00
2022-01-06, SELL EXECUTED, 3811.60
2022-01-06, Close, 3819.15
2022-01-06, BUY CREATE, 3819.15
2022-01-06, BUY EXECUTED, 3819.15
2022-01-06, Close, 3804.85
2022-01-0

In [27]:
df

,Open,High,Low,Close,Adj Close,Volume
2022-01-03 09:15:00+05:30,3748.350098,3809.850098,3745.449951,3800.149902,3800.149902,0
2022-01-03 10:15:00+05:30,3801.449951,3821.449951,3799.000000,3818.949951,3818.949951,395568
2022-01-03 11:15:00+05:30,3818.949951,3821.399902,3802.649902,3803.050049,3803.050049,255092
2022-01-03 12:15:00+05:30,3803.050049,3819.699951,3803.000000,3819.500000,3819.500000,206801
2022-01-03 13:15:00+05:30,3819.550049,3824.899902,3812.850098,3824.899902,3824.899902,272247
...,...,...,...,...,...,...
2022-03-08 11:15:00+05:30,3563.449951,3580.000000,3560.050049,3573.000000,3573.000000,365464
2022-03-08 12:15:00+05:30,3573.149902,3580.000000,3562.649902,3569.699951,3569.699951,325745
2022-03-08 13:15:00+05:30,3569.699951,3598.949951,3566.250000,3592.399902,3592.399902,372787
2022-03-08 14:15:00+05:30,3592.399902,3599.000000,3580.000000,3597.350098,3597.350098,181263
